### 네이버 연관 검색어 수집
- 정적(static) 웹페이지 데이터 수집 
- BeautifulSoup을 이용하여 HTML 문자열 데이터 parsing

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

#### 1. 웹페이지 분석 : URL

In [3]:
query = '삼성전자'
url = f'https://search.naver.com/search.naver?query={query}'

#### 2. request(URL) > response : str(html)

In [4]:
response = requests.get(url)
response

<Response [200]>

In [6]:
response.text[:200]

'<!doctype html> <html lang="ko"><head> <meta charset="utf-8"> <meta name="referrer" content="always">  <meta name="format-detection" content="telephone=no,address=no,email=no"> <meta name="viewport" c'

#### 3. str(html) > bs object

In [7]:
dom = BeautifulSoup(response.text, 'html.parser')
type(dom)

bs4.BeautifulSoup

#### 4. bs object > .select(css-selector), .select_one(css-selector) > str(text)

In [10]:
# dir(dom)
# help(dom.select) # 모든 엘리먼트
help(dom.select_one) # 맨 처음 엘리먼트

Help on method select_one in module bs4.element:

select_one(selector, namespaces=None, **kwargs) method of bs4.BeautifulSoup instance
    Perform a CSS selection operation on the current element.
    
    :param selector: A CSS selector.
    
    :param namespaces: A dictionary mapping namespace prefixes
       used in the CSS selector to namespace URIs. By default,
       Beautiful Soup will use the prefixes it encountered while
       parsing the document.
    
    :param kwargs: Keyword arguments to be passed into Soup Sieve's
       soupsieve.select() method.
    
    :return: A Tag.
    :rtype: bs4.element.Tag



In [18]:
elements = dom.select('.lst_related_srch > .item')
len(elements), elements[0]

(10,
 <li class="item"> <a class="keyword" href="?where=nexearch&amp;query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90%EC%A3%BC%EA%B0%80&amp;ie=utf8&amp;sm=tab_she&amp;qdt=0" onclick="return goOtherCR(this, 'a=rsk_top*q.list1&amp;r=1&amp;u=' + urlencode(this.href))"> <div class="tit">삼성전자주가</div> </a> </li>)

#### 5. str(text) > DataFrame

In [19]:
element = elements[0]
element

<li class="item"> <a class="keyword" href="?where=nexearch&amp;query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90%EC%A3%BC%EA%B0%80&amp;ie=utf8&amp;sm=tab_she&amp;qdt=0" onclick="return goOtherCR(this, 'a=rsk_top*q.list1&amp;r=1&amp;u=' + urlencode(this.href))"> <div class="tit">삼성전자주가</div> </a> </li>

In [20]:
keyword = element.select_one('.tit').text
keyword

'삼성전자주가'

In [23]:
keywords = []
for element in elements:
    keyword = element.select_one('.tit').text
    keywords.append(keyword)
keywords

['삼성전자주가',
 '삼성전자 배당금',
 '삼성전자서비스',
 '삼성전자주식',
 '오늘 삼성전자 주가',
 '삼성전자 배당금 지급일',
 '삼성전자몰',
 '삼성 전자레인지',
 '삼성전자 채용',
 '삼성전자 고객센터']

In [24]:
keywords = [element.select_one('.tit').text for element in elements]
keywords

['삼성전자주가',
 '삼성전자 배당금',
 '삼성전자서비스',
 '삼성전자주식',
 '오늘 삼성전자 주가',
 '삼성전자 배당금 지급일',
 '삼성전자몰',
 '삼성 전자레인지',
 '삼성전자 채용',
 '삼성전자 고객센터']

In [25]:
# 데이터 프레임 만들기
query, keywords

('삼성전자',
 ['삼성전자주가',
  '삼성전자 배당금',
  '삼성전자서비스',
  '삼성전자주식',
  '오늘 삼성전자 주가',
  '삼성전자 배당금 지급일',
  '삼성전자몰',
  '삼성 전자레인지',
  '삼성전자 채용',
  '삼성전자 고객센터'])

In [33]:
from datetime import datetime
datetime.now()

datetime.datetime(2023, 8, 25, 12, 19, 23, 542)

In [34]:
datetime.now().strftime('%Y-%m-%d %H:%M')

'2023-08-25 12:20'

In [35]:
now = datetime.now().strftime('%Y-%m-%d %H:%M')
now

'2023-08-25 12:20'

In [36]:
df = pd.DataFrame({'keyword':keywords})
df['query'] = query
df['datetime'] = now

In [37]:
df

,keyword,query,datetime
0,삼성전자주가,삼성전자,2023-08-25 12:20
1,삼성전자 배당금,삼성전자,2023-08-25 12:20
2,삼성전자서비스,삼성전자,2023-08-25 12:20
3,삼성전자주식,삼성전자,2023-08-25 12:20
4,오늘 삼성전자 주가,삼성전자,2023-08-25 12:20
5,삼성전자 배당금 지급일,삼성전자,2023-08-25 12:20
6,삼성전자몰,삼성전자,2023-08-25 12:20
7,삼성 전자레인지,삼성전자,2023-08-25 12:20
8,삼성전자 채용,삼성전자,2023-08-25 12:20
9,삼성전자 고객센터,삼성전자,2023-08-25 12:20


In [38]:
def naver_relate_keywords(query):
    
    url = f'https://search.naver.com/search.naver?query={query}'
    response = requests.get(url)
    dom = BeautifulSoup(response.text, 'html.parser')
    elements = dom.select('.lst_related_srch > .item')
    keywords = [element.select_one('.tit').text for element in elements]
    df = pd.DataFrame({'keyword':keywords})
    df['query'] = query
    df['datetime'] = datetime.now().strftime('%Y-%m-%d %H:%M')
    return df

In [42]:
df1 = naver_relate_keywords('삼성전자')

In [43]:
df2 = naver_relate_keywords('KT')

In [44]:
df3 = naver_relate_keywords('LG전자')

In [48]:
pd.concat([df1, df2, df3], axis=0, ignore_index=True)

,keyword,query,datetime
0,삼성전자주가,삼성전자,2023-08-25 12:24
1,삼성전자 배당금,삼성전자,2023-08-25 12:24
2,삼성전자서비스,삼성전자,2023-08-25 12:24
3,삼성전자주식,삼성전자,2023-08-25 12:24
4,오늘 삼성전자 주가,삼성전자,2023-08-25 12:24
5,삼성전자 배당금 지급일,삼성전자,2023-08-25 12:24
6,삼성전자몰,삼성전자,2023-08-25 12:24
7,삼성 전자레인지,삼성전자,2023-08-25 12:24
8,삼성전자 채용,삼성전자,2023-08-25 12:24
9,삼성전자 고객센터,삼성전자,2023-08-25 12:24
